In [108]:
import pandas as pd
import numpy as np
import spacy
sp_mod = spacy.load("en_core_web_sm")
import os, sys
import glob

In [109]:
def cell_expand(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [110]:
filenames = glob.glob('/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/del/*.story', recursive = True)
# filenames = filenames[0]
print(filenames)

['/Users/Mohit/Desktop/Work/Shreays Dataset/chuncker_raw_new_dataset/cnn/del/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story']


In [111]:
files = {} 
allstory_list = []
for filename in filenames: 
    with open(filename, "r") as file: 
        if filename in files: 
            continue 
        files[filename] = file.read() 
for filename, text in files.items(): 
    allstory_list.append(text)

In [112]:
# allstory_list

In [113]:
final_list = []
for i in range(len(allstory_list)):
    res_list = allstory_list[0].splitlines()
    res = [ele for ele in res_list if ele != []]
    res = [x for x in res if x]
    res = res[: len(res) - 8]
    final_list.append(res)
    # print(res)

In [114]:
# final_list

In [115]:
final_list = [['(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?']]

In [116]:
final_list

[['(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?']]

In [117]:
final_list[0][0]

'(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?'

In [118]:
text = text.strip()
doc = sp_mod(text)
sents = [elem for elem in doc.sents]

In [119]:
# IMP

for i in range(len(final_list)):
    for j in range(len(final_list[i])):
        new_sents = [] # for sentences with lenght greater than 5
        text = final_list[i][j]
        doc = sp_mod(text)
        # Tokenise the sentence
        sents = [elem for elem in doc.sents]
        # Idnetify the length of the sentence
        sents_len = [len(elem) for elem in doc.sents]
        
        # Ignore the tokenise setence with length less than 5 
        for k in range(len(sents_len)):
            if sents_len[k] > 5:
                new_sents.append(sents[k])
        
        # Assign to final list
        final_list[i][j] = new_sents
        

In [120]:
final_list

[[[Can a movie actually convince you to support torture?,
   Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment?,
   Can a movie truly cause you to view certain minority groups in a negative light?]]]

In [121]:
# stop123

In [122]:
# for i in range(len(final_list)):
#     for j in range(len(final_list[i])):
#         new_sents = [] # for sentences with lenght greater than 5
#         text = final_list[i][j]
#         doc = sp_mod(text)
#         # Tokenise the sentence
#         sents = [elem for elem in doc.sents]
#         # Idnetify the length of the sentence
#         sents_len = [len(elem) for elem in doc.sents]
        
#         # Ignore the tokenise setence with length less than 5 
#         for k in range(len(sents_len)):
#             if sents_len[k] > 5:
#                 new_sents.append(sents[k])
        
#         print(new_sents)
#         print(len(new_sents))
df = pd.DataFrame(columns=['sent_list'], index = range(1))
df['sent_list'][0] = new_sents

df['sent_len'] = None
for i in range(df.shape[0]):
    df['sent_len'][i] = len(df['sent_list'][i])
df = df[df['sent_len']>1]
df = df.reset_index(drop=True)

df['sent-1'] = None
df['sent-2'] = None

for i in range(df.shape[0]):

    sent_1_list = []
    sent_2_list = []
    for j in range(df['sent_len'][i]):

        sent_1_list.append(df['sent_list'][i][j])
        if j+1 < df['sent_len'][i]:
            sent_2_list.append(df['sent_list'][i][j+1])
    df['sent-1'][i] = sent_1_list
    df['sent-2'][i] = sent_2_list
    
    if len(df['sent-1'][i]) > len(df['sent-2'][i]):
        df['sent-1'][i].pop()


col_list = ['sent-1','sent-2']
df = cell_expand(df, lst_cols=col_list)

df['sent-1'] = df['sent-1'].astype('str')
df['sent-2'] = df['sent-2'].astype('str')
df['sent-1'].replace('', np.nan, inplace=True)
df.dropna(subset=['sent-1'], inplace=True)
df['sent-2'].replace('', np.nan, inplace=True)
df.dropna(subset=['sent-2'], inplace=True)
        

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [123]:
# for i in range(len(final_list)):
#     for j in range(len(final_list[i])):
#         new_sents = [] # for sentences with lenght greater than 5
#         text = final_list[i][j]
#         doc = sp_mod(text)
#         # Tokenise the sentence
#         sents = [elem for elem in doc.sents]
#         # Idnetify the length of the sentence
#         sents_len = [len(elem) for elem in doc.sents]
        
#         # Ignore the tokenise setence with length less than 5 
#         for k in range(len(sents_len)):
#             if sents_len[k] > 5:
#                 new_sents.append(sents[k])
        
#         print(new_sents)
#         print(len(new_sents))
#         df = pd.DataFrame(columns=['sent_list'], index = range(1))
#         df['sent_list'][0] = new_sents

#         df['sent_len'] = None
#         for i in range(df.shape[0]):
#             df['sent_len'][i] = len(df['sent_list'][i])
#         df = df[df['sent_len']>1]
#         df = df.reset_index(drop=True)
        
#         df['sent-1'] = None
#         df['sent-2'] = None

#         for i in range(df.shape[0]):

#             sent_1_list = []
#             sent_2_list = []
#             for j in range(df['sent_len'][i]):

#                 sent_1_list.append(df['sent_list'][i][j])
#                 if j+1 < df['sent_len'][i]:
#                     sent_2_list.append(df['sent_list'][i][j+1])
#             df['sent-1'][i] = sent_1_list
#             df['sent-2'][i] = sent_2_list
            
#             if len(df['sent-1'][i]) > len(df['sent-2'][i]):
#                 df['sent-1'][i].pop()

        
#         col_list = ['sent-1','sent-2']
#         df = cell_expand(df, lst_cols=col_list)

#         df['sent-1'] = df['sent-1'].astype('str')
#         df['sent-2'] = df['sent-2'].astype('str')
#         df['sent-1'].replace('', np.nan, inplace=True)
#         df.dropna(subset=['sent-1'], inplace=True)
#         df['sent-2'].replace('', np.nan, inplace=True)
#         df.dropna(subset=['sent-2'], inplace=True)
        

In [124]:
df

,sent_list,sent_len,sent-1,sent-2
0,"[(Can, a, movie, actually, convince, you, to, ...",3,Can a movie actually convince you to support t...,"Can a movie really persuade you that ""fracking..."
1,"[(Can, a, movie, actually, convince, you, to, ...",3,"Can a movie really persuade you that ""fracking...",Can a movie truly cause you to view certain mi...


In [125]:
# df['sent-2']

In [126]:
# final_list

In [127]:
# final_list[0][0]

In [128]:
df = pd.DataFrame(columns=['text'], index = range(len(final_list[0][0])))
df['text'] = final_list[0][0]
df.head()

,text
0,"(Can, a, movie, actually, convince, you, to, s..."
1,"(Can, a, movie, really, persuade, you, that, ""..."
2,"(Can, a, movie, truly, cause, you, to, view, c..."


In [129]:
type(final_list[0][0])

list

In [130]:
for i in range(len(final_list)):
    for j in range(len(final_list[i])):
        print(len(final_list[i][j]))
        print(final_list[i][j])
        if len(final_list[i][j]) > 2:
            
        # for k in range(len(final_list[i][j])):
        #     print(final_list[i][j][k])
        #     print("\n")

SyntaxError: unexpected EOF while parsing (<ipython-input-130-b68232bf6ffd>, line 9)